In [1]:
%load_ext autoreload
%autoreload 2

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
#export
from exp.nb_07 import *

<IPython.core.display.Javascript object>

[Jump_to lesson 11 video](https://course.fast.ai/videos/?lesson=11&t=235)

## Layerwise Sequential Unit Variance (LSUV)

Getting the MNIST data and a CNN

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

x_train, x_valid = normalize_to(x_train, x_valid)
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

nh, bs = 50, 512
c = y_train.max().item() + 1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

<IPython.core.display.Javascript object>

In [4]:
mnist_view = view_tfm(1, 28, 28)
cbfs = [
    Recorder,
    partial(AvgStatsCallback, accuracy),
    #         CudaCallback,
    partial(BatchTransformXCallback, mnist_view)
]

<IPython.core.display.Javascript object>

In [5]:
nfs = [8, 16, 32, 64, 64]

<IPython.core.display.Javascript object>

In [6]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni,
                              nf,
                              ks,
                              padding=ks // 2,
                              stride=stride,
                              bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)

    def forward(self, x):
        return self.relu(self.conv(x))

    @property
    def bias(self):
        return -self.relu.sub

    @bias.setter
    def bias(self, v):
        self.relu.sub = -v

    @property
    def weight(self):
        return self.conv.weight

<IPython.core.display.Javascript object>

In [7]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

<IPython.core.display.Javascript object>

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [8]:
run.fit(2, learn)

train: [1.44698625, tensor(0.5049)]
valid: [0.2342824462890625, tensor(0.9299)]
train: [0.18283890625, tensor(0.9434)]
valid: [0.15303546142578126, tensor(0.9508)]


<IPython.core.display.Javascript object>

Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [9]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

<IPython.core.display.Javascript object>

Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [10]:
#export
def get_batch(dl, run):
    run.xb, run.yb = next(iter(dl))
    for cb in run.cbs:
        cb.set_runner(run)
    run('begin_batch')
    return run.xb, run.yb

<IPython.core.display.Javascript object>

In [11]:
xb, yb = get_batch(data.train_dl, run)

<IPython.core.display.Javascript object>

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [12]:
sum([[1]], [2])

[2, 1]

<IPython.core.display.Javascript object>

In [13]:
#export
def find_modules(modules, cond):
    if cond(modules):
        return [modules]
    return sum([find_modules(module, cond) for module in modules.children()], [])


def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

<IPython.core.display.Javascript object>

In [14]:
mods = find_modules(learn.model, lambda o: isinstance(o, ConvLayer))

<IPython.core.display.Javascript object>

In [15]:
learn.model

Sequential(
  (0): ConvLayer(
    (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (relu): GeneralRelu()
  )
  (1): ConvLayer(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (2): ConvLayer(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (3): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (4): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (5): AdaptiveAvgPool2d(output_size=1)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

<IPython.core.display.Javascript object>

In [16]:
list(learn.model.children())

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 AdaptiveAvgPool2d(output_size=1),
 Lambda(),
 Linear(in_features=64, out_features=10, bias=True)]

<IPython.core.display.Javascript object>

In [17]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

<IPython.core.display.Javascript object>

This is a helper function to grab the mean and std of the output of a hooked layer.

In [18]:
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean, hook.std = d.mean().item(), d.std().item()

<IPython.core.display.Javascript object>

In [19]:
mdl = learn.model

<IPython.core.display.Javascript object>

So now we can look at the mean and std of the conv layers of our model.

In [20]:
with Hooks(mods, append_stat, is_forward=True) as hooks:
    mdl(xb)
    for hook in hooks:
        print(hook.mean, hook.std)

0.40883275866508484 0.8593772053718567
0.5693058967590332 0.9884846210479736
0.5329693555831909 0.94581538438797
0.5223042368888855 0.8221233487129211
0.30593863129615784 0.4924566149711609


<IPython.core.display.Javascript object>

We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [21]:
#export
def lsuv_module(m, xb):
    h = Hook(m, append_stat)

    while mdl(xb) is not None and abs(h.mean) > 1e-3:
        m.bias -= h.mean

    while mdl(xb) is not None and abs(h.std - 1) > 1e-3:
        m.weight.data /= h.std

    h.remove()
    return h.mean, h.std

<IPython.core.display.Javascript object>

We execute that initialization on all the conv layers in order:

In [22]:
for m in mods:
    print(lsuv_module(m, xb))

(0.06690089404582977, 1.0)
(0.01645469292998314, 1.0)
(0.11676739156246185, 1.0)
(0.14462140202522278, 0.9999999403953552)
(0.31877586245536804, 1.0000001192092896)


<IPython.core.display.Javascript object>

Note that the mean doesn't exactly stay at 0. since we change the standard deviation after by scaling the weight.

Then training is beginning on better grounds.

In [23]:
%time run.fit(2, learn)

train: [nan, tensor(0.1216)]
valid: [nan, tensor(0.0991)]
train: [nan, tensor(0.0986)]
valid: [nan, tensor(0.0991)]
CPU times: user 20.5 s, sys: 1.35 s, total: 21.8 s
Wall time: 11 s


<IPython.core.display.Javascript object>

LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [24]:
!python ../src/notebook2script.py 07a_LSUV-Imad.ipynb

Converted 07a_LSUV-Imad.ipynb to exp/nb_07a.py


<IPython.core.display.Javascript object>